In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
fdir = r'D:\non_2p\adam\2022_06_10_abb12'
fname = '2022_06_10_abb12'

## Conditions exercise 1: Detecting and loading data in different file formats.

The cell below defines the absolute file path (`epoc_data_path`) for the behavioral data. Currently we are loading in the .feather file, which is just another file format that can store tabular data. In the sample_data folder, we also have a csv version - try changing the behavioral data path to the csv version, then edit the conditional in the following cell to load the csv.

In [ ]:
epoc_data_path = os.path.join(fdir, f'{fname}_events.feather')
epoc_data_path

In [ ]:
if 'feather' in epoc_data_path:
    print('loading feather file')
    epoc_data = pd.read_feather(epoc_data_path)
elif 'csv' in epoc_data_path:
    print('loading csv file')
    epoc_data = pd.read_csv(epoc_data_path)
else:
    print('Not a valid file type')
    

In [ ]:
epoc_data.head()

The preview of the behavioral data table above doesn't really tell us how many behavioral conditions there are (event_id_char column). We can have python look through all entries in the event_id_char column and tell us what unique entries there are:

In [ ]:
event_conditions = list(epoc_data['event_id_char'].unique()) # grab unique condition names
event_conditions

## Conditions exercise 2: Checking contents of lists using conditionals and boolean operators

In [ ]:
if 'access_period' in event_conditions and 'lick' in event_conditions:
    print('Both conditions present')
elif 'access_period' in event_conditions or 'lick' in event_conditions:
    print('One out of the two conditions present')

## Pandas exercise 1: Exploring contents of a dataframe

In [ ]:
epoc_data.nunique() # provides count of unique entries in each column
epoc_data.info() # can identify if there is missing data

In [ ]:
epoc_data.head()

In [ ]:
epoc_data.tail()

In [ ]:
epoc_data.iloc[95:105]

In [ ]:
# we can also get a sense of how many events (y dimension) there are. If we use the method `shape`, the first dimension outputted is y and the 2nd in x.
epoc_data.shape

## Pandas exercise 2: Obtaining data from a dataframe column based on information in another column.

We are working with neural activity data that was synchronized to events that happened during the recording. Our behavioral dataframe contains event times (event_ts) for two different types of behavooral events (access_periods and licks). Here let's figure out how to extract event times for a given condition

In [ ]:
# grab event times for condition 1
event_one = epoc_data['event_ts'][epoc_data['event_id_char'] == 'access_period'].values
event_one

## Work on tseries data

## Pandas, math, and functions combined exercise: Correcting photometry traces with isosbestic channel

In [ ]:
data_path = os.path.join(fdir, f'{fname}_streams_session.feather')

data = pd.read_feather(data_path)
data

In [ ]:
fiber_id = '1' # 1 is NAcShell_medial; 2 is NAcShell_lateral

data_405 = data['control'][data['fiber_id'] == fiber_id].values
data_465 = data['signal'][data['fiber_id'] == fiber_id].values

tvec = data['time'][data['fiber_id'] == fiber_id].values 
tvec_zeroed = tvec - data['time'][0] # for plotting trace starting from time 0

fs = 1.0/np.mean(np.diff(tvec))

print(tvec_zeroed)

In [ ]:
def controlFit(control, signal):
    
    # GuPPY version
    # function to fit control channel to signal channel

    p = np.polyfit(control, signal, 1)
    arr = (p[0]*control)+p[1]
    return arr


def deltaFF(signal, control):
    
    # function to compute deltaF/F using fitted control channel and filtered signal channel

    res = np.subtract(signal, control) # numerator of F(t)-f0
    normData = np.divide(res, control) # (F(t)-f0)/F0
    normData = normData*100

    return normData

fitted_control = controlFit(data_405, data_465) # this function would be helpful if we wanted to correct signals from multiple fluorophore channels
corrected_data = deltaFF(data_465, fitted_control) # this function would be helpful everytime we needed to compute dF/F

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(tvec_zeroed, data_405)
plt.plot(tvec_zeroed, data_465)
plt.plot(tvec_zeroed, fitted_control)
plt.plot(tvec_zeroed, corrected_data)
plt.xlabel('Time (s)')
plt.ylabel('Fluorescence')
#plt.xlim([170, 400]) # we can zoom in to see how motion artifacts can be corrected for
plt.legend(['405', '470', 'fit curve', 'corrected']);

### Dictionary Exercise: Dividing event times up into their respective conditions. These event times can then be organized into a dictionary for storage purposes.

In [ ]:
# grab event times for all conditions via for loop
event_times_dict = {}
for event_name in epoc_data['event_id_char'].unique():
    event_times_dict[event_name] = epoc_data['event_ts'][epoc_data['event_id_char'] == event_name].values

event_times_dict

### Convert behavioral event times to samples: to prepare for indexing and snipping out trial activity windows.

#### Requires knowledge on: dictionaries and functions 

The data itself is not characterized by units of time, rather each item in the vector is a "sample" in the recording and does not hold any information on timing

On the other hand our behavioral events are in units of time. Accordingly, to extract trial snippits from the photometry data, we need to figure out which sample corresponds to a given event time

We can use this line of code : `np.argmin(abs(tvec - time))`

In [ ]:
print(f'First event from condition one: {event_one[0]}')

print(f"The sample that corresponds to {event_one[0]} is {np.argmin(abs(tvec - event_one[0]))}")

print(f"Sample {np.argmin(abs(tvec - event_one[0]))}'s time: {tvec[4224]}") # if dataset changes, replace number in tvec[4224] with calculated sample number

### Let's extract the activity trace from one trial for demonstration purposes

In [ ]:
# initialize start/end times for trial window
event_window_time = np.array([-1, 10])
event_window_samples = (event_window_time*fs).astype(int) # samples are integers

# generate vector the maps time onto samples in the event trial
trial_window_num_samples = event_window_samples[1] - event_window_samples[0]
tvec_trial = np.linspace(event_window_time[0], event_window_time[1], trial_window_num_samples)

# generate vector of every sample between start and end samples
# then offset to specific event's window by adding the event sample
trial_template_indices = np.arange(event_window_samples[0], event_window_samples[1])
trial_samples = trial_template_indices + np.argmin(abs(tvec - event_one[0]))

plt.plot(tvec_trial, corrected_data[trial_samples])

### Dictionary lesson: To analyze all trials, let's do some organization of the event conditions and trial times

In [ ]:
# turn the above code into a function
def get_tvec_sample(tvec, time):
    return np.argmin(abs(tvec - time))

event_samples_dict = {}
for condition_name in epoc_data['event_id_char'].unique(): # loop through conditions
    
    print(condition_name)
    tmp_list = []
    
    # go through each event and compute the sample/frame that it occurred on (b/c the list is currently in seconds)
    for event in event_times_dict[condition_name]: # loop through events
        tmp_list.append(get_tvec_sample(tvec, event))
        
    # once we go through converting each time to sample, and add to a list iteratively,
    # store that list to its corresponding condition in the dictionary
    event_samples_dict[condition_name] = tmp_list

In [ ]:
event_samples_dict['access_period']

### Numpy lesson: Let's analyze data from all trials

In [ ]:
# remove trials that have windows outside of entire recording, then extract data from each trial

data_trial_dict = {}
for condition_name in epoc_data['event_id_char'].unique(): # loop through conditions
    
    # initialize numpy array to populate with trial-extracted data
    tmp_trial_array = np.empty([len(event_samples_dict[condition_name]), trial_window_num_samples])
    
    # use trial index template, offset with event sample, and extract trial data
    for idx, event_sample in enumerate(event_samples_dict[condition_name]): # loop through events
        tmp_trial_array[idx,:] = corrected_data[trial_template_indices + event_sample]
    
    # once data are fully populated, add to dictionary
    data_trial_dict[condition_name] = tmp_trial_array

In [ ]:
cond_names = list(data_trial_dict.keys())

plt.imshow(data_trial_dict[cond_names[0]], aspect='auto')
plt.ylabel('Trial', fontsize=14)
plt.xlabel('Sample', fontsize=14)
cbar = plt.colorbar()
cbar.set_label('Fluorescence', fontsize=14)

In [ ]:
plt.plot(tvec_trial, np.mean(data_trial_dict[cond_names[0]], axis=0))
plt.plot(tvec_trial, np.mean(data_trial_dict[cond_names[1]], axis=0))
plt.ylabel('Fluorescence', fontsize=14)
plt.xlabel('Time', fontsize=14)
plt.legend(cond_names, fontsize=14)